In [1]:
import os
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from product_classification.data_processing.train_test_val_split import IterativeSplit, SimpleSplit 
from product_classification.data_processing.create_dataset import clean_dataset, get_merged_dataframe

/home/okteto/product_classification/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
path = "../data"
joined_tables = get_merged_dataframe(path)

2022-09-12 06:08:35,157 :: create_dataset/create_dataset.py/get_merged_dataframe :: INFO :: Create dataset
2022-09-12 06:09:33,128 :: create_dataset/create_dataset.py/get_merged_dataframe :: INFO :: Dataframe created
2022-09-12 06:09:33,129 :: create_dataset/create_dataset.py/get_merged_dataframe :: INFO :: Dataset shape: (470347, 7)


In [3]:
dataset = clean_dataset(joined_tables)

2022-09-12 06:09:33,749 :: create_dataset/create_dataset.py/clean_dataset :: INFO :: Duplicates removed
2022-09-12 06:09:33,751 :: create_dataset/create_dataset.py/clean_dataset :: INFO :: Dataset shape: (462695, 7)
/home/okteto/product_classification/product_classification/product_classification/data_processing/create_dataset.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["merchant_name"] = dataset["merchant_name"].astype("category")
/home/okteto/product_classification/product_classification/product_classification/data_processing/create_dataset.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

## Create train, test, validation set

In [4]:
min_category_count = 300
max_category_count = 5_000
split_size = 0.15
random_state = 42
simple_split = SimpleSplit(min_categories_threshold=min_category_count, max_categories_threshold=max_category_count)

In [5]:
simple_datasets = simple_split.execute(dataset=dataset, split_size=split_size, random_state=random_state)

2022-09-12 06:09:34,315 :: train_test_val_split/train_test_val_split.py/execute :: INFO :: Split dataset into train, test, val using train_test_split with stratification
2022-09-12 06:09:34,317 :: train_test_val_split/train_test_val_split.py/_filter_categories :: INFO :: Apply filter to remove categories with few examples
2022-09-12 06:09:34,358 :: train_test_val_split/train_test_val_split.py/_filter_categories :: INFO :: Removed categories: applis et jeux ,boutique cheques-cadeaux ,cigarettes ,garantie ,livraison ,logiciels ,telechargement de musique ,uncategorized
2022-09-12 06:09:34,359 :: train_test_val_split/train_test_val_split.py/_filter_categories :: INFO :: Dataset shape: (357320, 7)
2022-09-12 06:09:34,360 :: train_test_val_split/train_test_val_split.py/_downsampling_categories :: INFO :: Downsampling categories with too much examples
2022-09-12 06:09:34,396 :: train_test_val_split/train_test_val_split.py/_downsampling_categories :: INFO :: Dataset shape: (100888, 7)
2022-09-

In [6]:
for fname, dataframe in simple_datasets:
    setattr(simple_datasets, fname, dataframe.drop(columns=["product_description"]))

## Serialize datasets

In [7]:
import pickle

with open(f"{path}/datasets.pkl", "wb") as handle:
    pickle.dump(simple_datasets, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(f"{path}/multilabel_binarizer.pkl", "wb") as handle:
    pickle.dump(simple_split.multilabel_binarizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(f"{path}/pos_weight.pkl", "wb") as handle:
    pickle.dump(simple_split.pos_weight, handle, protocol=pickle.HIGHEST_PROTOCOL)